Import the dataset with `mongoimport --type csv -d heart_attack_risk_db  -c heart_attack_data --headerline --drop heart_attack_risk.csv`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

# Import MongoClient Key
from config import mongo_key

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(mongo_key)

In [3]:
# Confirm that our new database was created
print(mongo.list_database_names())

['COVID19_DB', 'CitiBike_DB', 'heart_attack_risk_db', 'admin', 'local']


In [4]:
# Assign the database to a variable name
db = mongo['heart_attack_risk_db']

In [5]:
# Review the collections in our new database
print(db.list_collection_names())

['heart_attack_data']


In [6]:
# Review a document in the collection
pprint(db.heart_attack_data.find_one())

{'Age': 67,
 'Alcohol Consumption': 0,
 'BMI': 31.25123273,
 'Blood Pressure': '158/88',
 'Cholesterol': 208,
 'Continent': 'South America',
 'Country': 'Argentina',
 'Diabetes': 0,
 'Diet': 'Average',
 'Exercise Hours Per Week': 4.168188835,
 'Family History': 0,
 'Heart Attack Risk': 0,
 'Heart Rate': 72,
 'Hemisphere': 'Southern Hemisphere',
 'Income': 261404,
 'Medication Use': 0,
 'Obesity': 0,
 'Patient ID': 'BMW7812',
 'Physical Activity Days Per Week': 0,
 'Previous Heart Problems': 0,
 'Sedentary Hours Per Day': 6.615001453,
 'Sex': 'Male',
 'Sleep Hours Per Day': 6,
 'Smoking': 1,
 'Stress Level': 9,
 'Triglycerides': 286,
 '_id': ObjectId('65cebe94cf813089ec40a416')}


In [16]:
# Assign the collection to a variable
heart_attack_records = db['heart_attack_data']

In [17]:
# Retrieve data from the collection
data_from_mongo = list(heart_attack_records.find())

In [18]:
# Convert the collection to a Pandas DataFrame
heart_attack_df = pd.DataFrame(data_from_mongo)

# Display the number of rows in the DataFrame
print('Number of rows:', len(heart_attack_df))

# Display the DataFrame
heart_attack_df.head()

Number of rows: 8763


,_id,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,65cebe94cf813089ec40a416,BMW7812,67,Male,208,158/88,72,0,0,1,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,65cebe94cf813089ec40a417,CZE1114,21,Male,389,165/93,98,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,65cebe94cf813089ec40a418,BNI9906,21,Female,324,174/99,72,1,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,65cebe94cf813089ec40a419,JLN3497,84,Male,383,163/100,73,1,1,1,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,65cebe94cf813089ec40a41a,GFO8847,66,Male,318,91/88,93,1,1,1,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0


In [19]:
# Confirm all columns are available
heart_attack_df.columns

Index(['_id', 'Patient ID', 'Age', 'Sex', 'Cholesterol', 'Blood Pressure',
       'Heart Rate', 'Diabetes', 'Family History', 'Smoking', 'Obesity',
       'Alcohol Consumption', 'Exercise Hours Per Week', 'Diet',
       'Previous Heart Problems', 'Medication Use', 'Stress Level',
       'Sedentary Hours Per Day', 'Income', 'BMI', 'Triglycerides',
       'Physical Activity Days Per Week', 'Sleep Hours Per Day', 'Country',
       'Continent', 'Hemisphere', 'Heart Attack Risk'],
      dtype='object')

In [20]:
Count = heart_attack_df['Heart Attack Risk'].count()
print(f'There are a total of {Count} values')

heart_attack_df['Heart Attack Risk'].value_counts()

There are a total of 8763 values


Heart Attack Risk
0    5624
1    3139
Name: count, dtype: int64

In [21]:
# Drop id column
heart_attack_df = heart_attack_df.drop('_id', axis=1)

In [22]:
pprint(heart_attack_df.columns.tolist())

['Patient ID',
 'Age',
 'Sex',
 'Cholesterol',
 'Blood Pressure',
 'Heart Rate',
 'Diabetes',
 'Family History',
 'Smoking',
 'Obesity',
 'Alcohol Consumption',
 'Exercise Hours Per Week',
 'Diet',
 'Previous Heart Problems',
 'Medication Use',
 'Stress Level',
 'Sedentary Hours Per Day',
 'Income',
 'BMI',
 'Triglycerides',
 'Physical Activity Days Per Week',
 'Sleep Hours Per Day',
 'Country',
 'Continent',
 'Hemisphere',
 'Heart Attack Risk']


In [23]:
# Save dataframe to csv
heart_attack_df.to_csv("heart_attack_risk_data.csv", index=False)

In [24]:
import csv
import json

# Convert CSV to JSON for dashboard
csvfile = open('heart_attack_risk_data.csv', 'r')
jsonfile = open('heart_attack_risk.json', 'w')

fieldnames = [
    'Patient ID', 'Age', 'Sex', 'Cholesterol', 'Blood Pressure',
    'Heart Rate', 'Diabetes', 'Family History', 'Smoking', 'Obesity',
    'Alcohol Consumption', 'Exercise Hours Per Week', 'Diet',
    'Previous Heart Problems', 'Medication Use', 'Stress Level',
    'Sedentary Hours Per Day', 'Income', 'BMI', 'Triglycerides',
    'Physical Activity Days Per Week', 'Sleep Hours Per Day', 'Country',
    'Continent', 'Hemisphere', 'Heart Attack Risk'
]

reader = csv.DictReader(csvfile, fieldnames)

# Create a list to hold the rows
data = []

# Skip the first row (header)
next(reader)

for row in reader:
    data.append(row)

# Use json.dump to write the entire list to the file
json.dump(data, jsonfile, indent=2)

# Close the files
csvfile.close()
jsonfile.close()